In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.0 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
[3] "icd9.sql"                "lab_tests.sql"          
[5] "lab_tests_taken_gen.sql" "lab_tests_taken.sql"    
[7] "lab_test_taken_tmpl"     "population.sql"         
[9] "vital_signs.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "icd9.sql")) %>%
    make_view_sql("icd9", "materialized view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from icd9 limit 10;
"
df <- dbGetQuery(con, sql)
df

hadm_id,subject_id,icustay_id,chf,afib,renal,liver,copd,cad,stroke,malignancy
100021,29971,252772,0,1,0,1,0,0,0,0
100028,53456,297261,0,1,0,0,0,0,0,0
100053,24912,217069,0,1,1,1,0,0,0,0
100099,82091,207036,1,1,0,0,1,1,0,0
100118,53749,202030,0,0,1,0,1,0,0,0
100132,19984,229201,0,1,0,0,0,0,0,0
100141,61215,243747,0,0,0,0,0,0,0,0
100154,22389,233463,1,0,0,0,1,0,0,1
100160,1601,249287,0,1,0,0,1,0,0,1
100191,6741,233457,0,0,0,0,0,0,1,0


In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE